# Preparing Libraries and Installations


In [1]:
%pip install bs4 selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install translators --upgrade

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/68.8 kB ? eta -:--:--
     ---------------------------------------- 68.8/68.8 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/56.4 kB ? eta -:--:--
   ---------------------------------------- 56.4/56.4 kB ? eta 0:00:00
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   --- ------------------------------------ 0.2/2.9 MB 4.6 MB/s eta 0:00:01
   ------ --------------------------------- 0.5/2.9 MB 5.2 MB/s eta 0:00:01
   ---------- ----------------------------- 0.8/2.9 MB 5.5 MB/s eta 0:00:01
   ------------- -------------------------- 1.0/2.9 MB 5.2 MB/s eta 0:00:01
   --------------- ------------------------ 1.1/2.9 MB 4.6 MB/s eta 0:00:01
   ---------------- ----------------------- 1.2/2.9 MB 4.3 MB/s eta 0:00:01
  

In [12]:
%pip install pymongo

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/484.6 kB ? eta -:--:--
   ---------------------- ----------------- 276.5/484.6 kB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 484.6/484.6 kB 4.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/307.7 kB ? eta -:--:--
   ---------------------------------------- 307.7/307.7 kB 9.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

from selenium.common.exceptions import (
    StaleElementReferenceException,
    TimeoutException,
    NoSuchElementException
)

from bs4 import BeautifulSoup

from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

import pandas as pd
import time

# Scraping the Website

In [2]:
# URL Ulasan dari Female Daily
base_url = "https://reviews.femaledaily.com/products/moisturizer/sun-protection-1/emina/sun-protection-spf-30-pa?cat=&cat_id=0&age_range=&skin_type=&skin_tone=&skin_undertone=&hair_texture=&hair_type=&order=oldest&page={}"
page = 1
reviews_count = 1

# Koneksi ke mongodb
username = "ayukrisna"
password = "bigdata"
host = "ayukrisna.c81wxan.mongodb.net"
dbname = "femaledaily_review"
uri = f"mongodb+srv://{username}:{password}@{host}/{dbname}?retryWrites=true&w=majority"

try:
    client = MongoClient(uri)
    db = client[dbname]
    collection = db['raw_data_duplicated']

    print("Successfully connected to MongoDB!")
except ConnectionFailure as e:
    print(f"Failed to connect to MongoDB: {e}")
    exit()

try:
    # Inisialisasi option chrome
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    # Inisialisasi WebDriver
    service = Service(executable_path = ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    #data frame
    df_data = pd.DataFrame(columns = ['date', 'username', 'age', 'skin_type', 'rating', 'recommended', 'review','usage_period', 'purchase_point', 'likes', 'comments'])

    # Scraping data
    while (1):
        # Buka URL
        url = base_url.format(page)
        driver.get(url)
        print(page)
        wait = WebDriverWait(driver, 10)

        # Ekstrak dan print title
        # print(driver.title)
        time.sleep(5) #tunggu 5 detik

        # Dapatkan list review
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        list_reviews = soup.find(class_='list-reviews')

        if not list_reviews:
            print("No reviews found on this page.")
            page += 1
            continue

        reviews = list_reviews.find_all(class_='review-card')
                       
        for review in reviews:
            #date
            date_element = review.find(class_='review-date')
            date = date_element.get_text(strip=True) if date_element else ""

            #left column
            left_col = review.find(class_='review-card-col left')
            username_element = left_col.find('p', class_='profile-username')
            age_element = left_col.find('p', class_='profile-age')
            skin_type_element = left_col.find('p', class_='profile-description')

            # Extract username, age, and skin type hanya jika ditemukan
            username = username_element.get_text(strip=True) if username_element else ""
            age = age_element.get_text(strip=True) if age_element else ""
            skin_type = skin_type_element.get_text(strip=True) if skin_type_element else ""

            #right column
            right_col = review.find(class_='review-card-col right')
            wrapper = right_col.find(class_='review-content-wrapper')

            #start ratings
            star_icons = wrapper.find_all('i', class_='icon-ic_big_star_full')
            rating = len(star_icons)

            #recommend
            recommendation_tag = wrapper.find('p', class_='recommend')
            if recommendation_tag:
                if 'recommends' in recommendation_tag.get_text():
                    recommended = "recommended"
                else:
                    recommended = "not recommended"
            else:
                recommended = ""

            #text review
            try:
                review_text_element = wrapper.find(class_='text-content')
                review_text = review_text_element.get_text(strip=True) if review_text_element else ""
            except (StaleElementReferenceException, NoSuchElementException):
                review_text = ""

            # Usage period and purchase point
            info_wrapper = wrapper.find(class_='information-wrapper')
            usage_period = ""
            purchase_point = ""
            if info_wrapper:
                usage_period_element = info_wrapper.find('span', string='Usage Period ')
                purchase_point_element = info_wrapper.find('span', string='Purchase Point ')

                # Extract usage period and purchase point hanya jika ditemukan
                if usage_period_element:
                    usage_period = usage_period_element.find_next('b').get_text(strip=True)
                if purchase_point_element:
                    purchase_point = purchase_point_element.find_next('b').get_text(strip=True)
            
            # Likes and dislikes
            comment_section = wrapper.find(class_='comment-section')
            likes = ""
            comments = ""
            if comment_section:
                likes_element = comment_section.find(class_= "like icon")
                comments_element = comment_section.find(class_= "chat icon")

                # Extract likes and comments hanya jika ditemukan
                if likes_element:
                    likes = likes_element.get_text(strip=True)
                if comments_element:
                    comments = comments_element.get_text(strip=True)

            # simpan ke frame
            df_data.loc[len(df_data)] = [date, username, age, skin_type, rating, recommended, review_text, usage_period, purchase_point, likes, comments]

            #simpan ke mongodb
            review_data = {
                'date': date,
                'username': username,
                'age': age,
                'skin_type': skin_type,
                'rating': rating,
                'recommended': recommended,
                'review': review_text,
                'usage_period': usage_period,
                'purchase_point': purchase_point,
                'likes': likes,
                'comments': comments
            }
            collection.insert_one(review_data)
            print(f"Successfully inserted {reviews_count}-reviews")
            reviews_count += 1

        # Memeriksa apakah ada button next page
        try:
            next_button = driver.find_element(By.ID, 'id_next_page')
            if 'paging-prev-text-inactive' in next_button.get_attribute('class'):
                break
        except NoSuchElementException:
            break

        page += 1
        time.sleep(2)

#exception
except WebDriverException as e:
    print(f"WebDriverException: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
except TimeoutException:
    print("The element was not clickable within the given time")
finally:
    driver.quit()

pd.set_option('display.max_colwidth', None)
df_data

Successfully connected to MongoDB!
412
Successfully inserted 1-reviews
Successfully inserted 2-reviews
Successfully inserted 3-reviews
Successfully inserted 4-reviews
Successfully inserted 5-reviews
Successfully inserted 6-reviews
Successfully inserted 7-reviews
Successfully inserted 8-reviews
Successfully inserted 9-reviews
Successfully inserted 10-reviews
413
Successfully inserted 11-reviews
Successfully inserted 12-reviews
Successfully inserted 13-reviews
Successfully inserted 14-reviews
Successfully inserted 15-reviews
Successfully inserted 16-reviews
Successfully inserted 17-reviews
Successfully inserted 18-reviews
Successfully inserted 19-reviews
Successfully inserted 20-reviews
414
Successfully inserted 21-reviews
Successfully inserted 22-reviews
Successfully inserted 23-reviews
Successfully inserted 24-reviews
Successfully inserted 25-reviews
Successfully inserted 26-reviews
Successfully inserted 27-reviews
Successfully inserted 28-reviews
Successfully inserted 29-reviews
Succe

,date,username,age,skin_type,rating,recommended,review,usage_period,purchase_point,likes,comments
0,17 May 2019,misslarasati,30 - 34,"Combination, Medium Light, Warm",4,,"biasa aja sih ini di gw, ga bikin breakout, cuma agak lengket aja menurut gw yg kulitnya oily, agak lama meresapnya.terus banyak isinya dgn hrg yg sangat affordable.bakal repurchase? maybe yes maybe no.",,,2,0
1,17 May 2019,nadhiroh,19 - 24,"Oily, Medium Dark, Warm",4,,"sunsceen ini teksturnya kental agak cair, untuk tipe kulit berminyak cocok aja menurutku, tapi kecepatan untuk menyerap dikulit butuh beberapa waktu dan perlu di diamkan. untuk pemakaiannya pastikan dulu wajah harus bersihbau dari produk ini enak sih menurutku dan jadi seger gitu kalo abis di aplikasikan",,,0,0
2,17 May 2019,luthfiahrnr_,19 - 24,"Combination, Medium, Neutral",5,,"beli sunscreen ini karna harganya affordable dan identik sama remaja. udah pake ini di botol ketiga saking sukanya karena gak bikin breakout, ga lengket, di muka juga ringan, dan ga bikin minyakan parah kalo uda beberapa jam",,,0,0
3,17 May 2019,fardew,19 - 24,"Combination, Medium, Warm",5,,"udah repurchase berkali kali, karena harganya yg pas di kantong ya kan drpd SB merk lainnyaa. ringan juga dipake, cepet ngeresap, packagingnya juga kyutt. Yang penting ga bikin wajah BO / jd kilang minyak",,,0,0
4,17 May 2019,septiliariscanova,19 - 24,"Normal, Medium, Neutral",5,,"Salah satu product sun protection yang affordable yang udah 3 kali di repurchase, gak ninggalin whitecast di muka sukakkk pokoknya, ampe udah ngeracunin ke sodara-sodara ku yang lain juga, ini andalan banget kalo kemana-mana kelupaan bisa stress wkwkwk",,,0,0
...,...,...,...,...,...,...,...,...,...,...,...
874,03 Jul 2019,helvyradhina,19 - 24,"Oily, Medium Light, Warm",3,,gasuka baunya kaya jeruk kematengan :( dan pas dipake ke muka tuh kaya berminya banget jd ga nyaman gitu. jd kupake buat tangan sama kaki aja klo lg kegiatan outdoor. lumayan buat menghindari belang hehe,,,1,0
875,03 Jul 2019,febella,19 - 24,"Oily, Light, Warm",5,,"Wow banget sih dengan harga segini! Ini tuh murah, tapi kualitas nggak murahan. Waktu awal-awal di-apply emang agak lama nyerepnya, tapi its okay untuk harga segini. Minus-nya cuma di baunya aja sih. Selebihnya, bagus banget nget ngettt! Sejak pakai ini, mukaku jadi nggak pernah terbakar lagi.",,,0,2
876,04 Jul 2019,divaputri96,19 - 24,"Oily, Medium Light, Cool",4,,"ini sunscreen pertama yg ak cb,meskioun bnyk jg yg blng bkl bkin kusam tp untng di ak gk,pas dipake tu enak ada snsasi dingin ,cepet nyerap,dn gk lengket.suka lah pkoknya.semakin hari semakin di pake juga ak ngeliat ad perubahan sma kulitku termasuk kalo siang bolong habis anter anak muka gak gitu gosong sih.klo pas dirumah aja ak tetep pake karena kan infonya panas dari alat masak juga pengaruh ke muka ya.mknya ak pake tiap hari pagi sore. hehe",,,0,0
877,04 Jul 2019,Febbytr,19 - 24,"Combination, Medium Light, Warm",5,,"sebagus itu sih, sunscreennya. gak lengket gitu.. tapi aku kurang suka wangi - wanginya gitu. tapi gak masalah sih yang penting ini bagus kalo kita mau berpergian jika cuacanya sedang panas. kurangnyaa, tubenya terlalu besar jadi suka susah buat dibawa di tas gitu kalo mau pergi pergi. dan ini awet gitu sih bagus lah pokoknya",,,0,0


# Delete Duplicate

In [4]:
import pandas as pd

all_data = list(collection.find())
df_data = pd.DataFrame(all_data)
df_data

,_id,date,username,age,skin_type,rating,recommended,review,usage_period,purchase_point,likes,comments
0,664ac96a7506cd60f11a6df3,14 hours ago,mtraandita,19 - 24,"Combination, Medium, Warm",3,not recommended,"Waktu smp aku pakai ss ini selama ±2 tahun, te...",More than 1 year,Shopee,0,0
1,664ac96c7506cd60f11a6df4,2 days ago,Miftahuljahra,19 - 24,"Combination, Medium Light, Cool",2,not recommended,Aku ga terlalu cocok pake produk ini terlebih ...,Less than 1 week,Shopee,1,0
2,664ac96c7506cd60f11a6df5,2 days ago,Marizka_nuril,19 - 24,"Combination, Medium, Neutral",3,not recommended,"Sunscreen pertama aku sewaktu kenal skincare, ...",3 months - 6 months,Shopee,0,0
3,664ac96c7506cd60f11a6df6,3 days ago,Denyul,30 - 34,"Combination, Medium Light, Cool",3,recommended,Teksturnya agak watery gituDiblendnya agak lum...,1 week - 1 month,Super Indo,0,0
4,664ac96d7506cd60f11a6df7,3 days ago,stephaniekumala,19 - 24,"Oily, Medium, Neutral",5,recommended,udah pake ini dari jaman SMA hahah seneng bang...,More than 1 year,Alfamart,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
14941,664b1e0e5e454cecdf3f5dda,03 Jul 2019,helvyradhina,19 - 24,"Oily, Medium Light, Warm",3,,gasuka baunya kaya jeruk kematengan :( dan pas...,,,1,0
14942,664b1e0e5e454cecdf3f5ddb,03 Jul 2019,febella,19 - 24,"Oily, Light, Warm",5,,Wow banget sih dengan harga segini! Ini tuh mu...,,,0,2
14943,664b1e0e5e454cecdf3f5ddc,04 Jul 2019,divaputri96,19 - 24,"Oily, Medium Light, Cool",4,,"ini sunscreen pertama yg ak cb,meskioun bnyk j...",,,0,0
14944,664b1e0e5e454cecdf3f5ddd,04 Jul 2019,Febbytr,19 - 24,"Combination, Medium Light, Warm",5,,"sebagus itu sih, sunscreennya. gak lengket git...",,,0,0


In [5]:
df_data.drop_duplicates(subset=['date', 'username', 'age', 'skin_type', 'rating', 'recommended', 'review','usage_period', 'purchase_point', 'likes', 'comments'], keep='first', inplace=True)
df_data

,_id,date,username,age,skin_type,rating,recommended,review,usage_period,purchase_point,likes,comments
0,664ac96a7506cd60f11a6df3,14 hours ago,mtraandita,19 - 24,"Combination, Medium, Warm",3,not recommended,"Waktu smp aku pakai ss ini selama ±2 tahun, te...",More than 1 year,Shopee,0,0
1,664ac96c7506cd60f11a6df4,2 days ago,Miftahuljahra,19 - 24,"Combination, Medium Light, Cool",2,not recommended,Aku ga terlalu cocok pake produk ini terlebih ...,Less than 1 week,Shopee,1,0
2,664ac96c7506cd60f11a6df5,2 days ago,Marizka_nuril,19 - 24,"Combination, Medium, Neutral",3,not recommended,"Sunscreen pertama aku sewaktu kenal skincare, ...",3 months - 6 months,Shopee,0,0
3,664ac96c7506cd60f11a6df6,3 days ago,Denyul,30 - 34,"Combination, Medium Light, Cool",3,recommended,Teksturnya agak watery gituDiblendnya agak lum...,1 week - 1 month,Super Indo,0,0
4,664ac96d7506cd60f11a6df7,3 days ago,stephaniekumala,19 - 24,"Oily, Medium, Neutral",5,recommended,udah pake ini dari jaman SMA hahah seneng bang...,More than 1 year,Alfamart,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
14331,664b1b6b5e454cecdf3f5b78,31 May 2019,anitaintan,25 - 29,"Combination, Medium, Warm",2,,Produk SS pertama yg aku coba.Beli ini karena ...,,,0,0
14332,664b1b6c5e454cecdf3f5b79,31 May 2019,HanPHLP,25 - 29,"Combination, Medium Light, Warm",2,,sebenernya nyoba beli sunscreen ini karena pen...,,,0,0
14333,664b1b6c5e454cecdf3f5b7a,31 May 2019,Putrialyam,19 - 24,"Combination, Medium, Warm",3,,Beli karena pas promo dan kebetulan sunscreen ...,,,0,0
14404,664b1bbb5e454cecdf3f5bc1,05 Jun 2019,jovriskae,18 and Under,"Combination, Medium, Cool",2,,"ini sunscreen pertamaku, awalnya pake ini karn...",,,0,0


In [5]:
collection = db['raw_data']
collection.insert_many(df_data.to_dict('records'))

InsertManyResult([ObjectId('664ac96a7506cd60f11a6df3'), ObjectId('664ac96c7506cd60f11a6df4'), ObjectId('664ac96c7506cd60f11a6df5'), ObjectId('664ac96c7506cd60f11a6df6'), ObjectId('664ac96d7506cd60f11a6df7'), ObjectId('664ac96d7506cd60f11a6df8'), ObjectId('664ac96d7506cd60f11a6df9'), ObjectId('664ac96d7506cd60f11a6dfa'), ObjectId('664ac96d7506cd60f11a6dfb'), ObjectId('664ac96d7506cd60f11a6dfc'), ObjectId('664ac97e7506cd60f11a6dfd'), ObjectId('664ac97e7506cd60f11a6dfe'), ObjectId('664ac97e7506cd60f11a6dff'), ObjectId('664ac97e7506cd60f11a6e00'), ObjectId('664ac97e7506cd60f11a6e01'), ObjectId('664ac97e7506cd60f11a6e02'), ObjectId('664ac97e7506cd60f11a6e03'), ObjectId('664ac97e7506cd60f11a6e04'), ObjectId('664ac97e7506cd60f11a6e05'), ObjectId('664ac97e7506cd60f11a6e06'), ObjectId('664ac98a7506cd60f11a6e07'), ObjectId('664ac98a7506cd60f11a6e08'), ObjectId('664ac98a7506cd60f11a6e09'), ObjectId('664ac98a7506cd60f11a6e0a'), ObjectId('664ac98a7506cd60f11a6e0b'), ObjectId('664ac98a7506cd60f11a6e

# Pre Processing

## Connect to MongoDB

In [52]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

# Koneksi ke mongodb
username = "ayukrisna"
password = "bigdata"
host = "ayukrisna.c81wxan.mongodb.net"
dbname = "femaledaily_review"
uri = f"mongodb+srv://{username}:{password}@{host}/{dbname}?retryWrites=true&w=majority"

try:
    client = MongoClient(uri)
    db = client[dbname]
    collection = db['data_berlabel_manual']

    print("Successfully connected to MongoDB!")
except ConnectionFailure as e:
    print(f"Failed to connect to MongoDB: {e}")
    exit()

Successfully connected to MongoDB!


In [53]:
import pandas as pd

all_data = list(collection.find())
df_data = pd.DataFrame(all_data)
df_data

,_id,date,username,age,skin_type,rating,recommended,review,usage_period,purchase_point,likes,comments,label,
0,664ac96c7506cd60f11a6df6,3 days ago,Denyul,30 - 34,"Combination, Medium Light, Cool",3,recommended,Teksturnya agak watery gituDiblendnya agak lum...,1 week - 1 month,Super Indo,0,0,netral,NaN
1,664ac96d7506cd60f11a6df7,3 days ago,stephaniekumala,19 - 24,"Oily, Medium, Neutral",5,recommended,udah pake ini dari jaman SMA hahah seneng bang...,More than 1 year,Alfamart,0,0,positif,NaN
2,664ac96d7506cd60f11a6df8,4 days ago,anakusumaayu,19 - 24,"Normal, Medium, Neutral",3,not recommended,"ini adalah suncreen pertamaku , dulu cocok ban...",6 months - 1 year,Guardian,0,0,netral,NaN
3,664ac96d7506cd60f11a6dfb,7 days ago,wiwiiii,19 - 24,"Combination, Medium Dark, Warm",1,not recommended,astaga ini tuh ss paling ngga banget dari semu...,1 month - 3 months,Tiktok,0,0,negatif,NaN
4,664ac97e7506cd60f11a6e01,10-May-24,Mah,25 - 29,"Combination, Medium Light, Cool",1,not recommended,Sekali pake besok nya ak langsung muncul jeraw...,1 week - 1 month,Official Brand Store,2,0,negatif,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14297,664b1b5f5e454cecdf3f5b6a,31-May-19,tzhanin,18 and Under,"Oily, Medium Light, Neutral",3,NaN,mayan bagus sih.. cuma klo make nya ketebelan ...,NaN,NaN,1,0,NaN,NaN
14298,664b1b5f5e454cecdf3f5b6c,31-May-19,annisa_maulida,19 - 24,"Combination, Medium Light, Neutral",3,NaN,beli ss ini karena emang harganya murah dan pa...,NaN,NaN,0,0,NaN,NaN
14299,664b1b5f5e454cecdf3f5b6d,31-May-19,syienasyam,19 - 24,"Oily, Medium Light, Neutral",3,NaN,beli ss ini karna kemarin lagi iseng aja liat ...,NaN,NaN,1,0,NaN,NaN
14300,664b1b6b5e454cecdf3f5b74,31-May-19,yansekarliana,18 and Under,"Dry, Light, Neutral",2,NaN,sebenernya produk ini ringan dikulit. lumayan ...,NaN,NaN,0,0,NaN,NaN


In [4]:
#hapus kolom kosong

df_data = df_data.drop(columns=[''])
df_data

,_id,date,username,age,skin_type,rating,recommended,review,usage_period,purchase_point,likes,comments,label
0,664ac96c7506cd60f11a6df6,3 days ago,Denyul,30 - 34,"Combination, Medium Light, Cool",3,recommended,Teksturnya agak watery gituDiblendnya agak lum...,1 week - 1 month,Super Indo,0,0,netral
1,664ac96d7506cd60f11a6df7,3 days ago,stephaniekumala,19 - 24,"Oily, Medium, Neutral",5,recommended,udah pake ini dari jaman SMA hahah seneng bang...,More than 1 year,Alfamart,0,0,positif
2,664ac96d7506cd60f11a6df8,4 days ago,anakusumaayu,19 - 24,"Normal, Medium, Neutral",3,not recommended,"ini adalah suncreen pertamaku , dulu cocok ban...",6 months - 1 year,Guardian,0,0,netral
3,664ac96d7506cd60f11a6dfb,7 days ago,wiwiiii,19 - 24,"Combination, Medium Dark, Warm",1,not recommended,astaga ini tuh ss paling ngga banget dari semu...,1 month - 3 months,Tiktok,0,0,negatif
4,664ac97e7506cd60f11a6e01,10-May-24,Mah,25 - 29,"Combination, Medium Light, Cool",1,not recommended,Sekali pake besok nya ak langsung muncul jeraw...,1 week - 1 month,Official Brand Store,2,0,negatif
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14297,664b1b5f5e454cecdf3f5b6a,31-May-19,tzhanin,18 and Under,"Oily, Medium Light, Neutral",3,NaN,mayan bagus sih.. cuma klo make nya ketebelan ...,NaN,NaN,1,0,NaN
14298,664b1b5f5e454cecdf3f5b6c,31-May-19,annisa_maulida,19 - 24,"Combination, Medium Light, Neutral",3,NaN,beli ss ini karena emang harganya murah dan pa...,NaN,NaN,0,0,NaN
14299,664b1b5f5e454cecdf3f5b6d,31-May-19,syienasyam,19 - 24,"Oily, Medium Light, Neutral",3,NaN,beli ss ini karna kemarin lagi iseng aja liat ...,NaN,NaN,1,0,NaN
14300,664b1b6b5e454cecdf3f5b74,31-May-19,yansekarliana,18 and Under,"Dry, Light, Neutral",2,NaN,sebenernya produk ini ringan dikulit. lumayan ...,NaN,NaN,0,0,NaN


## Change Date Format

In [7]:
#install library
%pip install python-dateutil

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
from datetime import datetime, timedelta
import re

reference_date = datetime(2024, 5, 20)

def preprocess_date(date_str):
    if 'hours ago' in date_str:
        match = re.search(r'(\d+)\s+hours\s+ago', date_str)
        if match:
            num_hours = int(match.group(1))
            return reference_date - timedelta(hours=num_hours)
    elif 'days ago' in date_str:
        match = re.search(r'(\d+)\s+days\s+ago', date_str)
        if match:
            num_days = int(match.group(1))
            return reference_date - timedelta(days=num_days)
    else:
        try:
            return datetime.strptime(date_str, '%d-%b-%y')
        except ValueError:
            return None

for index, row in df_data.iterrows():
    item_date = row['date']
    new_date = preprocess_date(item_date)
    if new_date:
        df_data.at[index, 'date'] = new_date

df_data

,_id,date,username,age,skin_type,rating,recommended,review,usage_period,purchase_point,likes,comments,label
0,664ac96c7506cd60f11a6df6,2024-05-17 00:00:00,Denyul,30 - 34,"Combination, Medium Light, Cool",3,recommended,Teksturnya agak watery gituDiblendnya agak lum...,1 week - 1 month,Super Indo,0,0,netral
1,664ac96d7506cd60f11a6df7,2024-05-17 00:00:00,stephaniekumala,19 - 24,"Oily, Medium, Neutral",5,recommended,udah pake ini dari jaman SMA hahah seneng bang...,More than 1 year,Alfamart,0,0,positif
2,664ac96d7506cd60f11a6df8,2024-05-16 00:00:00,anakusumaayu,19 - 24,"Normal, Medium, Neutral",3,not recommended,"ini adalah suncreen pertamaku , dulu cocok ban...",6 months - 1 year,Guardian,0,0,netral
3,664ac96d7506cd60f11a6dfb,2024-05-13 00:00:00,wiwiiii,19 - 24,"Combination, Medium Dark, Warm",1,not recommended,astaga ini tuh ss paling ngga banget dari semu...,1 month - 3 months,Tiktok,0,0,negatif
4,664ac97e7506cd60f11a6e01,2024-05-10 00:00:00,Mah,25 - 29,"Combination, Medium Light, Cool",1,not recommended,Sekali pake besok nya ak langsung muncul jeraw...,1 week - 1 month,Official Brand Store,2,0,negatif
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14297,664b1b5f5e454cecdf3f5b6a,2019-05-31 00:00:00,tzhanin,18 and Under,"Oily, Medium Light, Neutral",3,NaN,mayan bagus sih.. cuma klo make nya ketebelan ...,NaN,NaN,1,0,NaN
14298,664b1b5f5e454cecdf3f5b6c,2019-05-31 00:00:00,annisa_maulida,19 - 24,"Combination, Medium Light, Neutral",3,NaN,beli ss ini karena emang harganya murah dan pa...,NaN,NaN,0,0,NaN
14299,664b1b5f5e454cecdf3f5b6d,2019-05-31 00:00:00,syienasyam,19 - 24,"Oily, Medium Light, Neutral",3,NaN,beli ss ini karna kemarin lagi iseng aja liat ...,NaN,NaN,1,0,NaN
14300,664b1b6b5e454cecdf3f5b74,2019-05-31 00:00:00,yansekarliana,18 and Under,"Dry, Light, Neutral",2,NaN,sebenernya produk ini ringan dikulit. lumayan ...,NaN,NaN,0,0,NaN


In [9]:
collection = db['data_preprocessed']
collection.insert_many(df_data.to_dict('records'))

InsertManyResult([ObjectId('664ac96a7506cd60f11a6df3'), ObjectId('664ac96c7506cd60f11a6df4'), ObjectId('664ac96c7506cd60f11a6df5'), ObjectId('664ac96c7506cd60f11a6df6'), ObjectId('664ac96d7506cd60f11a6df7'), ObjectId('664ac96d7506cd60f11a6df8'), ObjectId('664ac96d7506cd60f11a6df9'), ObjectId('664ac96d7506cd60f11a6dfa'), ObjectId('664ac96d7506cd60f11a6dfb'), ObjectId('664ac96d7506cd60f11a6dfc'), ObjectId('664ac97e7506cd60f11a6dfd'), ObjectId('664ac97e7506cd60f11a6dfe'), ObjectId('664ac97e7506cd60f11a6dff'), ObjectId('664ac97e7506cd60f11a6e00'), ObjectId('664ac97e7506cd60f11a6e01'), ObjectId('664ac97e7506cd60f11a6e02'), ObjectId('664ac97e7506cd60f11a6e03'), ObjectId('664ac97e7506cd60f11a6e04'), ObjectId('664ac97e7506cd60f11a6e05'), ObjectId('664ac97e7506cd60f11a6e06'), ObjectId('664ac98a7506cd60f11a6e07'), ObjectId('664ac98a7506cd60f11a6e08'), ObjectId('664ac98a7506cd60f11a6e09'), ObjectId('664ac98a7506cd60f11a6e0a'), ObjectId('664ac98a7506cd60f11a6e0b'), ObjectId('664ac98a7506cd60f11a6e

In [10]:
df_data['label'].value_counts()

label
netral     500
negatif    500
positif    500
Name: count, dtype: int64

## Translating

In [15]:
%pip install -U deep-translator

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
from deep_translator import GoogleTranslator

collection = db['data_preprocessed']

all_data = list(collection.find())
df_data = pd.DataFrame(all_data)

translator_in_to_en = GoogleTranslator(source='id', target='en')
translator_en_to_in = GoogleTranslator(source='en', target='id')

def translate_reviews(review):
    # Translate from Indonesian to English
    review_in_en = translator_in_to_en.translate(review)
    # Translate from English back to Indonesian
    review_back_in_in = translator_en_to_in.translate(review_in_en)
    return review_in_en, review_back_in_in

i = 0
for idx, row in df_data.iterrows():
    review = row['review']
    review_in_english, review_back_in_indo = translate_reviews(review)
    collection.update_one(
        {'_id': row['_id']},
        {'$set': {'review': review_back_in_indo}}
    )
    i += 1
    print("Data " + str(i) + " updated successfully.")

Data 1 updated successfully.
Data 2 updated successfully.
Data 3 updated successfully.
Data 4 updated successfully.
Data 5 updated successfully.
Data 6 updated successfully.
Data 7 updated successfully.
Data 8 updated successfully.
Data 9 updated successfully.
Data 10 updated successfully.
Data 11 updated successfully.
Data 12 updated successfully.
Data 13 updated successfully.
Data 14 updated successfully.
Data 15 updated successfully.
Data 16 updated successfully.
Data 17 updated successfully.
Data 18 updated successfully.
Data 19 updated successfully.
Data 20 updated successfully.
Data 21 updated successfully.
Data 22 updated successfully.
Data 23 updated successfully.
Data 24 updated successfully.
Data 25 updated successfully.
Data 26 updated successfully.
Data 27 updated successfully.
Data 28 updated successfully.
Data 29 updated successfully.
Data 30 updated successfully.
Data 31 updated successfully.
Data 32 updated successfully.
Data 33 updated successfully.
Data 34 updated suc

ChunkedEncodingError: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))

In [12]:
import pandas as pd
from deep_translator import GoogleTranslator

collection = db['data_preprocessed']

all_data = list(collection.find())
df_data = pd.DataFrame(all_data)

translator_in_to_en = GoogleTranslator(source='id', target='en')
translator_en_to_in = GoogleTranslator(source='en', target='id')

def translate_reviews(review):
    # Translate from Indonesian to English
    review_in_en = translator_in_to_en.translate(review)
    # Translate from English back to Indonesian
    review_back_in_in = translator_en_to_in.translate(review_in_en)
    return review_in_en, review_back_in_in

i = 10872
for idx, row in df_data.iloc[10872:].iterrows():
    review = row['review']
    review_in_english, review_back_in_indo = translate_reviews(review)
    collection.update_one(
        {'_id': row['_id']},
        {'$set': {'review': review_back_in_indo}}
    )
    i += 1
    print("Data " + str(i) + " updated successfully.")

Data 10873 updated successfully.
Data 10874 updated successfully.
Data 10875 updated successfully.
Data 10876 updated successfully.
Data 10877 updated successfully.
Data 10878 updated successfully.
Data 10879 updated successfully.
Data 10880 updated successfully.
Data 10881 updated successfully.
Data 10882 updated successfully.
Data 10883 updated successfully.
Data 10884 updated successfully.
Data 10885 updated successfully.
Data 10886 updated successfully.
Data 10887 updated successfully.
Data 10888 updated successfully.
Data 10889 updated successfully.
Data 10890 updated successfully.
Data 10891 updated successfully.
Data 10892 updated successfully.
Data 10893 updated successfully.
Data 10894 updated successfully.
Data 10895 updated successfully.
Data 10896 updated successfully.
Data 10897 updated successfully.
Data 10898 updated successfully.
Data 10899 updated successfully.
Data 10900 updated successfully.
Data 10901 updated successfully.
Data 10902 updated successfully.
Data 10903

KeyboardInterrupt: 

In [6]:
import pandas as pd
from deep_translator import GoogleTranslator

collection = db['data_preprocessed']

all_data = list(collection.find())
df_data = pd.DataFrame(all_data)

translator_in_to_en = GoogleTranslator(source='id', target='en')
translator_en_to_in = GoogleTranslator(source='en', target='id')

def translate_reviews(review):
    # Translate from Indonesian to English
    review_in_en = translator_in_to_en.translate(review)
    # Translate from English back to Indonesian
    review_back_in_in = translator_en_to_in.translate(review_in_en)
    return review_in_en, review_back_in_in

i = 14208
for idx, row in df_data.iloc[14208:].iterrows():
    review = row['review']
    review_in_english, review_back_in_indo = translate_reviews(review)
    collection.update_one(
        {'_id': row['_id']},
        {'$set': {'review': review_back_in_indo}}
    )
    i += 1
    print("Data " + str(i) + " updated successfully.")

Data 14209 updated successfully.
Data 14210 updated successfully.
Data 14211 updated successfully.
Data 14212 updated successfully.
Data 14213 updated successfully.
Data 14214 updated successfully.
Data 14215 updated successfully.
Data 14216 updated successfully.
Data 14217 updated successfully.
Data 14218 updated successfully.
Data 14219 updated successfully.
Data 14220 updated successfully.
Data 14221 updated successfully.
Data 14222 updated successfully.
Data 14223 updated successfully.
Data 14224 updated successfully.
Data 14225 updated successfully.
Data 14226 updated successfully.
Data 14227 updated successfully.
Data 14228 updated successfully.
Data 14229 updated successfully.
Data 14230 updated successfully.
Data 14231 updated successfully.
Data 14232 updated successfully.
Data 14233 updated successfully.
Data 14234 updated successfully.
Data 14235 updated successfully.
Data 14236 updated successfully.
Data 14237 updated successfully.
Data 14238 updated successfully.
Data 14239

In [6]:
import pandas as pd
from deep_translator import GoogleTranslator

collection = db['data_preprocessed']

all_data = list(collection.find())
df_data = pd.DataFrame(all_data)

translator_in_to_en = GoogleTranslator(source='id', target='en')
translator_en_to_in = GoogleTranslator(source='en', target='id')

def translate_reviews(review):
    # Translate from Indonesian to English
    review_in_en = translator_in_to_en.translate(review)
    # Translate from English back to Indonesian
    review_back_in_in = translator_en_to_in.translate(review_in_en)
    return review_in_en, review_back_in_in

i = 14251
for idx, row in df_data.iloc[14251:].iterrows():
    review = row['review']
    review_in_english, review_back_in_indo = translate_reviews(review)
    collection.update_one(
        {'_id': row['_id']},
        {'$set': {'review': review_back_in_indo}}
    )
    i += 1
    print("Data " + str(i) + " updated successfully.")

Data 14252 updated successfully.
Data 14253 updated successfully.
Data 14254 updated successfully.
Data 14255 updated successfully.
Data 14256 updated successfully.
Data 14257 updated successfully.
Data 14258 updated successfully.
Data 14259 updated successfully.
Data 14260 updated successfully.
Data 14261 updated successfully.
Data 14262 updated successfully.
Data 14263 updated successfully.
Data 14264 updated successfully.
Data 14265 updated successfully.
Data 14266 updated successfully.
Data 14267 updated successfully.
Data 14268 updated successfully.
Data 14269 updated successfully.
Data 14270 updated successfully.
Data 14271 updated successfully.
Data 14272 updated successfully.
Data 14273 updated successfully.
Data 14274 updated successfully.
Data 14275 updated successfully.
Data 14276 updated successfully.
Data 14277 updated successfully.
Data 14278 updated successfully.
Data 14279 updated successfully.
Data 14280 updated successfully.
Data 14281 updated successfully.
Data 14282

## Text Cleaning and Text Folding

In [2]:
%pip install emoji --upgrade

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/431.4 kB ? eta -:--:--
    --------------------------------------- 10.2/431.4 kB ? eta -:--:--
    --------------------------------------- 10.2/431.4 kB ? eta -:--:--
   -- ------------------------------------ 30.7/431.4 kB 146.3 kB/s eta 0:00:03
   --- ----------------------------------- 41.0/431.4 kB 196.9 kB/s eta 0:00:02
   ----- --------------------------------- 61.4/431.4 kB 252.2 kB/s eta 0:00:02
   -------- ------------------------------ 92.2/431.4 kB 291.5 kB/s eta 0:00:02
   ------------ ------------------------- 143.4/431.4 kB 405.9 kB/s eta 0:00:01
   ---------------------- --------------- 256.0/431.4 kB 655.4 kB/s eta 0:00:01
   ---------------------- --------------- 256.0/431.4 kB 655.4 kB/s eta 0:00:01
   ---------------------- --------------- 256.0/431.4 kB 655.4 kB/s eta 0:00:01
   ---------------------- --------------- 256.0/431.4 kB 655.4 kB/s

In [54]:
# Get collection
import pandas as pd

collection = db['data_preprocessed']

all_data = list(collection.find())
df_data = pd.DataFrame(all_data)
df_data

,_id,date,username,age,skin_type,rating,recommended,review,usage_period,purchase_point,likes,comments,label
0,664ac96a7506cd60f11a6df3,2024-05-19 10:00:00,mtraandita,19 - 24,"Combination, Medium, Warm",3,not recommended,"Waktu SMP saya pakai SS ini ±2 tahun, lalu say...",More than 1 year,Shopee,0,0,netral
1,664ac96c7506cd60f11a6df4,2024-05-18 00:00:00,Miftahuljahra,19 - 24,"Combination, Medium Light, Cool",2,not recommended,"Aku kurang terlalu suka pakai produk ini, apal...",Less than 1 week,Shopee,1,0,negatif
2,664ac96c7506cd60f11a6df5,2024-05-18 00:00:00,Marizka_nuril,19 - 24,"Combination, Medium, Neutral",3,not recommended,Tabir surya pertamaku saat pertama kali mengap...,3 months - 6 months,Shopee,0,0,netral
3,664ac96c7506cd60f11a6df6,2024-05-17 00:00:00,Denyul,30 - 34,"Combination, Medium Light, Cool",3,recommended,Teksturnya agak cair. Perpaduannya cukup mengu...,1 week - 1 month,Super Indo,0,0,netral
4,664ac96d7506cd60f11a6df7,2024-05-17 00:00:00,stephaniekumala,19 - 24,"Oily, Medium, Neutral",5,recommended,"Aku pakai ini dari SMA hahaha, seneng banget p...",More than 1 year,Alfamart,0,0,positif
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14297,664b1b6b5e454cecdf3f5b78,2019-05-31 00:00:00,anitaintan,25 - 29,"Combination, Medium, Warm",2,NaN,Produk SS pertama yang saya coba. Saya membeli...,NaN,NaN,0,0,NaN
14298,664b1b6c5e454cecdf3f5b79,2019-05-31 00:00:00,HanPHLP,25 - 29,"Combination, Medium Light, Warm",2,NaN,Sebenarnya saya mencoba membeli tabir surya in...,NaN,NaN,0,0,NaN
14299,664b1b6c5e454cecdf3f5b7a,2019-05-31 00:00:00,Putrialyam,19 - 24,"Combination, Medium, Warm",3,NaN,Dibeli karena sedang promosi dan kebetulan sun...,NaN,NaN,0,0,NaN
14300,664b1bbb5e454cecdf3f5bc1,2019-06-05 00:00:00,jovriskae,18 and Under,"Combination, Medium, Cool",2,NaN,"Ini tabir surya pertamaku, awalnya aku pakai i...",NaN,NaN,0,0,NaN


In [59]:
import re
import emoji

def clean_text(text):
    # Remove emojis
    text = emoji.demojize(text, language='id')
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove hashtags and special characters except punctuation, numbers, and letters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    return text

df_data['review'] = df_data['review'].apply(clean_text)

collection = db['final_preprocessed']
collection.insert_many(df_data.to_dict('records'))

df_data

,_id,date,username,age,skin_type,rating,recommended,review,usage_period,purchase_point,likes,comments,label
0,664ac96a7506cd60f11a6df3,2024-05-19 10:00:00,mtraandita,19 - 24,"Combination, Medium, Warm",3,not recommended,waktu smp saya pakai ss ini tahun lalu saya b...,More than 1 year,Shopee,0,0,netral
1,664ac96c7506cd60f11a6df4,2024-05-18 00:00:00,Miftahuljahra,19 - 24,"Combination, Medium Light, Cool",2,not recommended,aku kurang terlalu suka pakai produk ini apala...,Less than 1 week,Shopee,1,0,negatif
2,664ac96c7506cd60f11a6df5,2024-05-18 00:00:00,Marizka_nuril,19 - 24,"Combination, Medium, Neutral",3,not recommended,tabir surya pertamaku saat pertama kali mengap...,3 months - 6 months,Shopee,0,0,netral
3,664ac96c7506cd60f11a6df6,2024-05-17 00:00:00,Denyul,30 - 34,"Combination, Medium Light, Cool",3,recommended,teksturnya agak cair perpaduannya cukup mengur...,1 week - 1 month,Super Indo,0,0,netral
4,664ac96d7506cd60f11a6df7,2024-05-17 00:00:00,stephaniekumala,19 - 24,"Oily, Medium, Neutral",5,recommended,aku pakai ini dari sma hahaha seneng banget pa...,More than 1 year,Alfamart,0,0,positif
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14297,664b1b6b5e454cecdf3f5b78,2019-05-31 00:00:00,anitaintan,25 - 29,"Combination, Medium, Warm",2,NaN,produk ss pertama yang saya coba saya membeli ...,NaN,NaN,0,0,NaN
14298,664b1b6c5e454cecdf3f5b79,2019-05-31 00:00:00,HanPHLP,25 - 29,"Combination, Medium Light, Warm",2,NaN,sebenarnya saya mencoba membeli tabir surya in...,NaN,NaN,0,0,NaN
14299,664b1b6c5e454cecdf3f5b7a,2019-05-31 00:00:00,Putrialyam,19 - 24,"Combination, Medium, Warm",3,NaN,dibeli karena sedang promosi dan kebetulan sun...,NaN,NaN,0,0,NaN
14300,664b1bbb5e454cecdf3f5bc1,2019-06-05 00:00:00,jovriskae,18 and Under,"Combination, Medium, Cool",2,NaN,ini tabir surya pertamaku awalnya aku pakai in...,NaN,NaN,0,0,NaN


## Remove Stopwords

In [3]:
%pip install sastrawi

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/209.7 kB ? eta -:--:--
   - -------------------------------------- 10.2/209.7 kB ? eta -:--:--
   - -------------------------------------- 10.2/209.7 kB ? eta -:--:--
   ------- ------------------------------- 41.0/209.7 kB 326.8 kB/s eta 0:00:01
   ------- ------------------------------- 41.0/209.7 kB 326.8 kB/s eta 0:00:01
   ----------------- --------------------- 92.2/209.7 kB 403.5 kB/s eta 0:00:01
   ----------------------------- -------- 163.8/209.7 kB 577.4 kB/s eta 0:00:01
   -------------------------------------  204.8/209.7 kB 689.9 kB/s eta 0:00:01
   -------------------------------------- 209.7/209.7 kB 579.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [65]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

def load_custom_stopwords(csv_path):
    df_stopwords = pd.read_csv(csv_path, header=None)
    return df_stopwords[0].tolist()

stopword_factory = StopWordRemoverFactory()
sastrawi_stopwords = stopword_factory.get_stop_words()

custom_stopwords_path = 'stopwordbahasa.csv'
custom_stopwords = load_custom_stopwords(custom_stopwords_path)

combined_stopwords = list(set(sastrawi_stopwords + custom_stopwords))
stopword_dictionary = ArrayDictionary(combined_stopwords)
stopword_remover = StopWordRemover(stopword_dictionary)


def stopword(text):
    text = stopword_remover.remove(text)
    return text

df_data['review'] = df_data['review'].apply(stopword)

for index, row in df_data.iterrows():
    review_id = row['_id']
    review_text = row['review']
    collection.update_one(
        {'_id': row['_id']},
        {'$set': {'review': review_text}}
    )

df_data

,_id,date,username,age,skin_type,rating,recommended,review,usage_period,purchase_point,likes,comments,label
0,664ac96a7506cd60f11a6df3,2024-05-19 10:00:00,mtraandita,19 - 24,"Combination, Medium, Warm",3,not recommended,smp pakai ss berhenti covid coba pakai breako...,More than 1 year,Shopee,0,0,netral
1,664ac96c7506cd60f11a6df4,2024-05-18 00:00:00,Miftahuljahra,19 - 24,"Combination, Medium Light, Cool",2,not recommended,suka pakai produk mukaku kulitnya berminyak pa...,Less than 1 week,Shopee,1,0,negatif
2,664ac96c7506cd60f11a6df5,2024-05-18 00:00:00,Marizka_nuril,19 - 24,"Combination, Medium, Neutral",3,not recommended,tabir surya pertamaku mengaplikasikannya suka ...,3 months - 6 months,Shopee,0,0,netral
3,664ac96c7506cd60f11a6df6,2024-05-17 00:00:00,Denyul,30 - 34,"Combination, Medium Light, Cool",3,recommended,teksturnya cair perpaduannya menguras tenaga m...,1 week - 1 month,Super Indo,0,0,netral
4,664ac96d7506cd60f11a6df7,2024-05-17 00:00:00,stephaniekumala,19 - 24,"Oily, Medium, Neutral",5,recommended,pakai sma seneng banget suka banget solusinya ...,More than 1 year,Alfamart,0,0,positif
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14297,664b1b6b5e454cecdf3f5b78,2019-05-31 00:00:00,anitaintan,25 - 29,"Combination, Medium, Warm",2,NaN,produk ss coba membeli membaca review bagus pa...,NaN,NaN,0,0,NaN
14298,664b1b6c5e454cecdf3f5b79,2019-05-31 00:00:00,HanPHLP,25 - 29,"Combination, Medium Light, Warm",2,NaN,mencoba membeli tabir surya penasaran sayangny...,NaN,NaN,0,0,NaN
14299,664b1b6c5e454cecdf3f5b7a,2019-05-31 00:00:00,Putrialyam,19 - 24,"Combination, Medium, Warm",3,NaN,dibeli promosi sunblock pakai habis kemasannya...,NaN,NaN,0,0,NaN
14300,664b1bbb5e454cecdf3f5bc1,2019-06-05 00:00:00,jovriskae,18 and Under,"Combination, Medium, Cool",2,NaN,tabir surya pertamaku pakai temanku kulitku bu...,NaN,NaN,0,0,NaN


## Stemming

In [66]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

def stemming(text):
    text = stemmer.stem(text)
    return text

df_data['review'] = df_data['review'].apply(stemming)

i = 0
for index, row in df_data.iterrows():
    review_id = row['_id']
    review_text = row['review']
    collection.update_one(
        {'_id': row['_id']},
        {'$set': {'review': review_text}}
    )
    i += 1
    print("Data " + str(i) + " updated successfully.")

df_data

Data 1 updated successfully.
Data 2 updated successfully.
Data 3 updated successfully.
Data 4 updated successfully.
Data 5 updated successfully.
Data 6 updated successfully.
Data 7 updated successfully.
Data 8 updated successfully.
Data 9 updated successfully.
Data 10 updated successfully.
Data 11 updated successfully.
Data 12 updated successfully.
Data 13 updated successfully.
Data 14 updated successfully.
Data 15 updated successfully.
Data 16 updated successfully.
Data 17 updated successfully.
Data 18 updated successfully.
Data 19 updated successfully.
Data 20 updated successfully.
Data 21 updated successfully.
Data 22 updated successfully.
Data 23 updated successfully.
Data 24 updated successfully.
Data 25 updated successfully.
Data 26 updated successfully.
Data 27 updated successfully.
Data 28 updated successfully.
Data 29 updated successfully.
Data 30 updated successfully.
Data 31 updated successfully.
Data 32 updated successfully.
Data 33 updated successfully.
Data 34 updated suc

,_id,date,username,age,skin_type,rating,recommended,review,usage_period,purchase_point,likes,comments,label
0,664ac96a7506cd60f11a6df3,2024-05-19 10:00:00,mtraandita,19 - 24,"Combination, Medium, Warm",3,not recommended,smp pakai ss henti covid coba pakai breakoutny...,More than 1 year,Shopee,0,0,netral
1,664ac96c7506cd60f11a6df4,2024-05-18 00:00:00,Miftahuljahra,19 - 24,"Combination, Medium Light, Cool",2,not recommended,suka pakai produk muka kulit minyak pakai jera...,Less than 1 week,Shopee,1,0,negatif
2,664ac96c7506cd60f11a6df5,2024-05-18 00:00:00,Marizka_nuril,19 - 24,"Combination, Medium, Neutral",3,not recommended,tabir surya pertama aplikasi suka wangi kuat t...,3 months - 6 months,Shopee,0,0,netral
3,664ac96c7506cd60f11a6df6,2024-05-17 00:00:00,Denyul,30 - 34,"Combination, Medium Light, Cool",3,recommended,tekstur cair padu uras tenaga muka panas guna ...,1 week - 1 month,Super Indo,0,0,netral
4,664ac96d7506cd60f11a6df7,2024-05-17 00:00:00,stephaniekumala,19 - 24,"Oily, Medium, Neutral",5,recommended,pakai sma neng banget suka banget solusi ringa...,More than 1 year,Alfamart,0,0,positif
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14297,664b1b6b5e454cecdf3f5b78,2019-05-31 00:00:00,anitaintan,25 - 29,"Combination, Medium, Warm",2,NaN,produk ss coba beli baca review bagus pakai bb...,NaN,NaN,0,0,NaN
14298,664b1b6c5e454cecdf3f5b79,2019-05-31 00:00:00,HanPHLP,25 - 29,"Combination, Medium Light, Warm",2,NaN,coba beli tabir surya penasaran sayang coba ku...,NaN,NaN,0,0,NaN
14299,664b1b6c5e454cecdf3f5b7a,2019-05-31 00:00:00,Putrialyam,19 - 24,"Combination, Medium, Warm",3,NaN,beli promosi sunblock pakai habis kemas lucu p...,NaN,NaN,0,0,NaN
14300,664b1bbb5e454cecdf3f5bc1,2019-06-05 00:00:00,jovriskae,18 and Under,"Combination, Medium, Cool",2,NaN,tabir surya pertama pakai teman kulit butuh ta...,NaN,NaN,0,0,NaN


## Normalisasi Tambahan

In [69]:
import re
import pandas as pd

collection = db['final_preprocessed'] 

all_data = list(collection.find())
df_data = pd.DataFrame(all_data)
df_data = df_data[df_data['label'].notna() & (df_data['label'] != '')]

def replace_sunscreen_variations(text):
    replacement_dict = {
        'sun screen': 'sunscreen',
        'ss': 'sunscreen',
        'tabir surya': 'sunscreen',
        'sunblock': 'sunscreen',
        'sun block': 'sunscreen',
        'make': 'pakai'
    }
    
    pattern = re.compile(r'\b(' + '|'.join(map(re.escape, replacement_dict.keys())) + r')\b', re.IGNORECASE)
    
    return pattern.sub(lambda x: replacement_dict[x.group().lower()], text)

df_data['review'] = df_data['review'].apply(replace_sunscreen_variations)

for index, row in df_data.iterrows():
    review_id = row['_id']
    review_text = row['review']
    collection.update_one(
        {'_id': row['_id']},
        {'$set': {'review': review_text}}
    )

df_data

,_id,date,username,age,skin_type,rating,recommended,review,usage_period,purchase_point,likes,comments,label
0,664ac96a7506cd60f11a6df3,2024-05-19 10:00:00,mtraandita,19 - 24,"Combination, Medium, Warm",3,not recommended,smp pakai sunscreen henti covid coba pakai bre...,More than 1 year,Shopee,0,0,netral
1,664ac96c7506cd60f11a6df4,2024-05-18 00:00:00,Miftahuljahra,19 - 24,"Combination, Medium Light, Cool",2,not recommended,suka pakai produk muka kulit minyak pakai jera...,Less than 1 week,Shopee,1,0,negatif
2,664ac96c7506cd60f11a6df5,2024-05-18 00:00:00,Marizka_nuril,19 - 24,"Combination, Medium, Neutral",3,not recommended,sunscreen pertama aplikasi suka wangi kuat tek...,3 months - 6 months,Shopee,0,0,netral
3,664ac96c7506cd60f11a6df6,2024-05-17 00:00:00,Denyul,30 - 34,"Combination, Medium Light, Cool",3,recommended,tekstur cair padu uras tenaga muka panas guna ...,1 week - 1 month,Super Indo,0,0,netral
4,664ac96d7506cd60f11a6df7,2024-05-17 00:00:00,stephaniekumala,19 - 24,"Oily, Medium, Neutral",5,recommended,pakai sma neng banget suka banget solusi ringa...,More than 1 year,Alfamart,0,0,positif
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2979,664ad6907506cd60f11a799c,2021-01-20 00:00:00,najmahnf,18 and Under,"Combination, Medium, Warm",1,not recommended,coba sunscreen putus beli emina suka sunscreen...,1 week - 1 month,Traditional market,0,0,negatif
2982,664ad6917506cd60f11a799f,2021-01-20 00:00:00,qodharrahma,19 - 24,"Combination, Medium Light, Neutral",2,not recommended,sunscreen pertama smp mampir toko kosmetik pen...,More than 1 year,Shopee,6,0,negatif
2989,664ad69c7506cd60f11a79a6,2021-01-20 00:00:00,seull_,18 and Under,"Oily, Medium, Neutral",1,not recommended,bikin kulit kusam minyak bikin jerawat sob har...,Less than 1 week,Beauty Studio,0,0,negatif
2999,664ad6a87506cd60f11a79b0,2021-01-19 00:00:00,widyaref,19 - 24,"Normal, Medium Light, Neutral",2,not recommended,gips putih lengket wajah kulit kering enak min...,3 months - 6 months,Gift,0,0,negatif


## Tokenization (tidak digunakan)

In [4]:
%pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [19]:
import nltk

nltk.download('punkt')

# Word Tokenization
def tokenize(text):
    word_tokens = nltk.word_tokenize(text)
    return word_tokens

i = 0
for document in collection.find():
    review_id = document['_id']
    review_text = document['review']
    # Tokenize the review text
    tokenized_review = tokenize(review_text)
    collection.update_one(
        {'_id': review_id},
        {'$set': {'tokenized_review': tokenized_review}}
    )
    i += 1
    print("Data " + str(i) + " updated successfully.")

[nltk_data] Downloading package punkt to C:\Users\Ayu
[nltk_data]     Krisna\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


Data 1 updated successfully.
Data 2 updated successfully.
Data 3 updated successfully.
Data 4 updated successfully.
Data 5 updated successfully.
Data 6 updated successfully.
Data 7 updated successfully.
Data 8 updated successfully.
Data 9 updated successfully.
Data 10 updated successfully.
Data 11 updated successfully.
Data 12 updated successfully.
Data 13 updated successfully.
Data 14 updated successfully.
Data 15 updated successfully.
Data 16 updated successfully.
Data 17 updated successfully.
Data 18 updated successfully.
Data 19 updated successfully.
Data 20 updated successfully.
Data 21 updated successfully.
Data 22 updated successfully.
Data 23 updated successfully.
Data 24 updated successfully.
Data 25 updated successfully.
Data 26 updated successfully.
Data 27 updated successfully.
Data 28 updated successfully.
Data 29 updated successfully.
Data 30 updated successfully.
Data 31 updated successfully.
Data 32 updated successfully.
Data 33 updated successfully.
Data 34 updated suc

## Penghapusan Kolom (Tidak dipakai)

In [6]:
import pandas as pd

collection = db['data_preprocessed2'] 
new_collection = db['data_preprocessed_final'] 

all_data = list(collection.find())
df_data = pd.DataFrame(all_data)

delete_columns=['date','username','age','skin_type','rating','recommended','review','usage_period','purchase_point','likes','comments']

df_data = df_data.drop(columns=delete_columns)

new_collection.insert_many(df_data.to_dict('records'))

InsertManyResult([ObjectId('664ac96a7506cd60f11a6df3'), ObjectId('664ac96c7506cd60f11a6df4'), ObjectId('664ac96c7506cd60f11a6df5'), ObjectId('664ac96c7506cd60f11a6df6'), ObjectId('664ac96d7506cd60f11a6df7'), ObjectId('664ac96d7506cd60f11a6df8'), ObjectId('664ac96d7506cd60f11a6df9'), ObjectId('664ac96d7506cd60f11a6dfa'), ObjectId('664ac96d7506cd60f11a6dfb'), ObjectId('664ac96d7506cd60f11a6dfc'), ObjectId('664ac97e7506cd60f11a6dfd'), ObjectId('664ac97e7506cd60f11a6dfe'), ObjectId('664ac97e7506cd60f11a6dff'), ObjectId('664ac97e7506cd60f11a6e00'), ObjectId('664ac97e7506cd60f11a6e01'), ObjectId('664ac97e7506cd60f11a6e02'), ObjectId('664ac97e7506cd60f11a6e03'), ObjectId('664ac97e7506cd60f11a6e04'), ObjectId('664ac97e7506cd60f11a6e05'), ObjectId('664ac97e7506cd60f11a6e06'), ObjectId('664ac98a7506cd60f11a6e07'), ObjectId('664ac98a7506cd60f11a6e08'), ObjectId('664ac98a7506cd60f11a6e09'), ObjectId('664ac98a7506cd60f11a6e0a'), ObjectId('664ac98a7506cd60f11a6e0b'), ObjectId('664ac98a7506cd60f11a6e

## Pelaksanaan TF-IDF

In [107]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd

collection = db['final_preprocessed'] 

all_data = list(collection.find())
df_data = pd.DataFrame(all_data)

df_data = df_data[df_data['label'].notna() & (df_data['label'] != '')]

reviews = df_data['review']
labels = df_data['label']


tfidf_vectorizer = TfidfVectorizer(min_df=0.01, max_df=0.9, max_features=2000)

X_tfidf = tfidf_vectorizer.fit_transform(reviews)

y = labels.values

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.1, random_state=42)

# Output shapes
print("Shape of TF-IDF Matrix:", X_tfidf.shape)
print("Shape of Training Set:", X_train.shape)
print("Shape of Test Set:", X_test.shape)
print("Shape of Label Array:", y.shape)

Shape of TF-IDF Matrix: (1500, 259)
Shape of Training Set: (1350, 259)
Shape of Test Set: (150, 259)
Shape of Label Array: (1500,)


In [108]:
import numpy as np

# Get feature names (words) from the vectorizer
feature_names = np.array(tfidf_vectorizer.get_feature_names_out())

# Get the maximum TF-IDF score for each word
max_tfidf_scores = X_tfidf.max(axis=0).toarray().flatten()

# Sort the words by their maximum TF-IDF scores in descending order
sorted_indices = max_tfidf_scores.argsort()[::-1]

# Get the top 10 words and their scores
top_10_words = feature_names[sorted_indices[:10]]
top_10_scores = max_tfidf_scores[sorted_indices[:10]]

# Print the top 10 words with their TF-IDF scores
for word, score in zip(top_10_words, top_10_scores):
    print(f"Word: {word}, TF-IDF Score: {score}")


Word: aman, TF-IDF Score: 1.0
Word: encer, TF-IDF Score: 0.829194464381387
Word: kelas, TF-IDF Score: 0.8291363864929407
Word: rias, TF-IDF Score: 0.8066153024740275
Word: breakout, TF-IDF Score: 0.8062744479341588
Word: kalang, TF-IDF Score: 0.7917425591474395
Word: wajah, TF-IDF Score: 0.7645721737497141
Word: oke, TF-IDF Score: 0.7597884479965759
Word: tangan, TF-IDF Score: 0.7466958081250152
Word: habis, TF-IDF Score: 0.7430531833552931


In [109]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Train the Multinomial Naive Bayes model
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = nb_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Output evaluation results
print("Accuracy:", accuracy)
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)


Accuracy: 0.5866666666666667

Classification Report:
               precision    recall  f1-score   support

     negatif       0.46      0.63      0.54        41
      netral       0.50      0.43      0.46        56
     positif       0.83      0.72      0.77        53

    accuracy                           0.59       150
   macro avg       0.60      0.59      0.59       150
weighted avg       0.61      0.59      0.59       150


Confusion Matrix:
 [[26 14  1]
 [25 24  7]
 [ 5 10 38]]


In [84]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

collection = db['final_preprocessed'] 

all_data = list(collection.find())
df_data = pd.DataFrame(all_data)

def load_custom_stopwords(csv_path):
    df_stopwords = pd.read_csv(csv_path, header=None)
    return df_stopwords[0].tolist()

stopword_factory = StopWordRemoverFactory()
sastrawi_stopwords = stopword_factory.get_stop_words()

custom_stopwords_path = 'stopwordbahasa.csv'
custom_stopwords = load_custom_stopwords(custom_stopwords_path)

combined_stopwords = list(set(sastrawi_stopwords + custom_stopwords))
stopword_dictionary = ArrayDictionary(combined_stopwords)
stopword_remover = StopWordRemover(stopword_dictionary)

def stopword(text):
    text = stopword_remover.remove(text)
    return text

df_data['review'] = df_data['review'].apply(stopword)

for index, row in df_data.iterrows():
    review_id = row['_id']
    review_text = row['review']
    collection.update_one(
        {'_id': row['_id']},
        {'$set': {'review': review_text}}
    )

df_data

,_id,date,username,age,skin_type,rating,recommended,review,usage_period,purchase_point,likes,comments,label
0,664ac96a7506cd60f11a6df3,2024-05-19 10:00:00,mtraandita,19 - 24,"Combination, Medium, Warm",3,not recommended,smp sunscreen henti covid coba breakout parah ...,More than 1 year,Shopee,0,0,netral
1,664ac96c7506cd60f11a6df4,2024-05-18 00:00:00,Miftahuljahra,19 - 24,"Combination, Medium Light, Cool",2,not recommended,suka produk muka kulit minyak jerawat langsung...,Less than 1 week,Shopee,1,0,negatif
2,664ac96c7506cd60f11a6df5,2024-05-18 00:00:00,Marizka_nuril,19 - 24,"Combination, Medium, Neutral",3,not recommended,sunscreen aplikasi suka wangi kuat tekstur gel...,3 months - 6 months,Shopee,0,0,netral
3,664ac96c7506cd60f11a6df6,2024-05-17 00:00:00,Denyul,30 - 34,"Combination, Medium Light, Cool",3,recommended,tekstur cair padu uras tenaga muka panas aroma...,1 week - 1 month,Super Indo,0,0,netral
4,664ac96d7506cd60f11a6df7,2024-05-17 00:00:00,stephaniekumala,19 - 24,"Oily, Medium, Neutral",5,recommended,sma neng banget suka banget solusi ringan muda...,More than 1 year,Alfamart,0,0,positif
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14297,664b1b6b5e454cecdf3f5b78,2019-05-31 00:00:00,anitaintan,25 - 29,"Combination, Medium, Warm",2,NaN,produk ss coba beli baca review bagus bb cream...,NaN,NaN,0,0,NaN
14298,664b1b6c5e454cecdf3f5b79,2019-05-31 00:00:00,HanPHLP,25 - 29,"Combination, Medium Light, Warm",2,NaN,coba beli tabir surya penasaran sayang coba ku...,NaN,NaN,0,0,NaN
14299,664b1b6c5e454cecdf3f5b7a,2019-05-31 00:00:00,Putrialyam,19 - 24,"Combination, Medium, Warm",3,NaN,beli promosi sunblock habis kemas lucu praktis...,NaN,NaN,0,0,NaN
14300,664b1bbb5e454cecdf3f5bc1,2019-06-05 00:00:00,jovriskae,18 and Under,"Combination, Medium, Cool",2,NaN,tabir surya teman kulit butuh tabir surya beli...,NaN,NaN,0,0,NaN
